# Semi-structured eval: Long-context

We will test retrival of table information from the `Semi-structured Reports` dataset using various methods.

In [ ]:
%pip install -U langchain langsmith langchain_benchmarks
%pip install -U anthropic openai

## Dataset

In [1]:
import os

from langchain_benchmarks import registry
from langchain_benchmarks.rag.tasks.semi_structured_reports import get_file_names

# Task
task = registry["Semi-structured Reports"]

# Files used
paths = list(get_file_names())
files = [str(p) for p in paths]

### TODO: Replace when dataset is updated
dir = "/Users/rlm/Desktop/Eval_Sets/semi_structured_reports/"
files = [dir + f for f in os.listdir(dir) if f.endswith(".pdf")]

## Load

In [2]:
from langchain.document_loaders import PyPDFLoader

texts = []
for fi in files:
    loader = PyPDFLoader(fi)
    pdf_pages = loader.load()
    texts.extend(pdf_pages)

texts = [t.page_content for t in texts]
text_string = " /// New Document /// ".join(texts)

## Run

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.chat_models import ChatAnthropic
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough


def create_chain(model):
    # Prompt template
    template = """Answer the question based only on the following context, which can include text and tables:
    {context}
    Question: {question}
    """
    prompt = ChatPromptTemplate.from_template(template)

    chain = (
        {
            "context": lambda x: text_string,
            "question": RunnablePassthrough(),
        }
        | prompt
        | model
        | StrOutputParser()
    )

    return chain


# OAI 128k
model = ChatOpenAI(temperature=0, model="gpt-4-1106-preview")
chain_oai_128k = create_chain(model)

# Anthropic 100k
model = ChatAnthropic(temperature=0, model="claude-2")
chain_claude = create_chain(model)

## Eval

In [6]:
import uuid
from langchain.smith import RunEvalConfig
from langsmith.client import Client

eval_config = RunEvalConfig(
    evaluators=["cot_qa"],
)


def run_eval(chain, eval_run_name):
    """
    Run eval
    """
    client = Client()
    test_run = client.run_on_dataset(
        ### TODO: Replace with public dataset
        dataset_name="Semi-Structured-Eval-v5",
        llm_or_chain_factory=lambda: (lambda x: x["question"]) | chain,
        evaluation=eval_config,
        verbose=True,
        project_name=eval_run_name,
    )


# Experiments
chain_map = {
    "oai_128k": chain_oai_128k,
    # "claude2_100k_v2": chain_claude,
}

run_id = str(uuid.uuid4())
for project_name, chain in chain_map.items():
    run_eval(chain, project_name + "_" + run_id)

View the evaluation results for project 'oai_128k_d364769e-4327-4f73-be4a-010495456c34' at:
https://smith.langchain.com/o/1fa8b1f4-fcb9-4072-9aa9-983e35ad61b8/projects/p/21333311-270b-44fa-83b6-608c89eb962b?eval=true

View all tests for Dataset Semi-Structured-Eval-v5 at:
https://smith.langchain.com/o/1fa8b1f4-fcb9-4072-9aa9-983e35ad61b8/datasets/2759f13d-c0c0-4d60-a8cf-0ce204750642
[------------------------------------------------->] 25/25
 Eval quantiles:
                                          inputs.question  \
count                                                  25   
unique                                                 25   
top     What is Datadog's Non-GAAP gross margin for th...   
freq                                                    1   
mean                                                  NaN   
std                                                   NaN   
min                                                   NaN   
25%                                               